In [1]:
import matplotlib.pyplot as plt
import pyspeckit 

from astropy.io import fits
from astropy.wcs import WCS

from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

from spectral_cube import SpectralCube
from spectral_cube import Projection

Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).


g++ -std=c++11   -Ofast -fopenmp -DNDEBUG -DHAVE_INLINE -MMD -MP -fPIC   -c -o pwind_geom.o pwind_geom.cpp


In file included from pwind_geom.cpp:2:
pwind_util.H:9:10: fatal error: gsl/gsl_sf_erf.h: No such file or directory
    9 | #include <gsl/gsl_sf_erf.h>
      |          ^~~~~~~~~~~~~~~~~~
compilation terminated.
make: *** [<builtin>: pwind_geom.o] Error 1


In [2]:
pos_cloudc1 = SkyCoord('17:46:21.3048683891', '-28:35:33.1211282499', unit=(u.hourangle, u.deg))
pos_cloudc2 = SkyCoord('17:46:18.3316118680', '-28:34:48.4717811920', unit=(u.hourangle, u.deg))
pos_cloudd = SkyCoord('17:46:22.6563371259', '-28:33:27.5405071803', unit=(u.hourangle, u.deg))
pos_filament = SkyCoord('17:46:20.9063719501', '-28:37:51.6942550990', unit=(u.hourangle, u.deg))

In [4]:
rad_cloudc1 = 15*u.arcsec
rad_cloudc2 = 15*u.arcsec
rad_cloudd = 30*u.arcsec
rad_filament = 1.5*u.arcmin

In [3]:
cube = SpectralCube.read('/orange/adamginsburg/ACES/mosaics/cubes/HNCO_7m12mTP_CubeMosaic.fits')

File may have been truncated: actual file length (989520002880) is smaller than the expected size (989520004800)
Could not memory map array with mode='readonly', falling back to mode='denywrite', which means that the array will be read-only


In [7]:
from spectral_cube import SpectralCube
from regions import CircleSkyRegion

# Create a circular region around pos_cloudc1 with the given radius
circle_region = CircleSkyRegion(center=pos_cloudc1, radius=rad_cloudc1)

# Extract the subcube around pos_cloudc1 with the given radius
subcube = cube.subcube_from_regions([circle_region], minimize=False)

# Compute the average spectrum along the spectral axis
average_spectrum = subcube.mean(axis=(1, 2))

This function (<function BaseSpectralCube.mean at 0x145e1be7b520>) requires loading the entire cube into memory and may therefore be slow.


In [ ]:
plt.plot(average_spectrum)

<CircleSkyRegion(center=<SkyCoord (ICRS): (ra, dec) in deg
    (266.58877028, -28.59253365)>, radius=15.0 arcsec)>